In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [3]:
# submission csv 파일 경로를 넣어주세요.
submission_files = [] 

submission_df = [pd.read_csv(file) for file in submission_files]

image_ids = submission_df[0]['image_id'].tolist()

annotation = '/opt/ml/dataset/test.json'
coco = COCO(annotation)

IndexError: list index out of range

In [3]:
prediction_strings = []
file_names = []

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        #boxes, scores, labels = nms(boxes_list, scores_list, labels_list,iou_thr=0.5)                       # NMS ensemble
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=0.5)     # WBF ensemble

        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [4]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

# 저장 경로를 넣어주세요.
# 현재 코드에서는 utils 디렉토리의 submission_ensemble.csv로 저장됩니다.
submission.to_csv('/opt/level2/level2_objectdetection_cv-level2-cv-17/mmdetection/configs/_trash_/_base_/utils/submission_ensemble.csv')        
submission.head()

,PredictionString,image_id
0,7 0.9999401569366455 217.22557067871094 47.987...,test/0000.jpg
1,5 0.9988875389099121 127.82943725585938 0.0364...,test/0001.jpg
2,1 0.9978135824203491 287.3660583496094 308.768...,test/0002.jpg
3,9 0.9301331043243408 59.82720947265625 140.238...,test/0003.jpg
4,1 0.9995791912078857 203.21682739257812 251.33...,test/0004.jpg
